Hi everybody , this little snippet will show you how to use the Twitter API and do the sentiment analysis for the each tweet found. We'll use different Python libs to make this exercise. 

The first thing you need to do is import the necessary libraries to use. In this case we'll need the TWEEPY in order to make a twitter connection, TEXTBLOB to run a sentiment analysis, TIME and DATETIME to control a pause (I explain this point better bellow) and record the time that the register was inserted in a database and MYSQL.CONNECTOR in order to connect and use the MySQL.

In [ ]:
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import datetime
import mysql.connector

After import the libs, we'll develop a function that makes a connection, passing the parameters to mysql connect method.

In [ ]:
connection = mysql.connector.connect(host='localhost',
                             database='database_name',
                             user='user',
                             password='password')

After that, we need to create a cursor passing the necessary insert statement

In [ ]:
cursor = connection.cursor(prepared=True)
        sql_insert_query = '''INSERT INTO `tweets`.`tweets_table`(`Date`,`Author`, `Text`, `Translate`, `Sentiment`, `Sensation`)VALUES(%s,%s,%s,%s,%s,%s)'''


Now the cursor is instantiated and the record found is commited in the database

In [ ]:
insert_tuple = (p_date,p_author, p_text, p_translate, p_sentiment,p_sensation)
        result = cursor.execute(sql_insert_query, insert_tuple)
        connection.commit()

To each entry commited, a message is displayed.

In [ ]:
print ("Record inserted successfully into tweets_table table")

I case of an error, a try.. except handles, makes a rollback and shows the error message

In [ ]:
except mysql.connector.Error as error :
        connection.rollback()
        print("Failed to insert into MySQL table {}".format(error))

In [ ]:
At the end, a finally block close the cursor and database connection

In [ ]:
finally:
        #closing database connection.
        if(connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

Below, the first part of the code completed:

In [ ]:
import tweepy
from tweepy import OAuthHandler
import timeimport pandas as pd
from textblob import TextBlob
import datetime
import mysql.connector

def insertInTable(p_date,p_author, p_text, p_translate, p_sentiment,p_sensation):
    try:
        connection = mysql.connector.connect(host='localhost',
                             database='databaase_name',
                             user='user',
                             password='password')
        cursor = connection.cursor(prepared=True)
        sql_insert_query = '''INSERT INTO `tweets`.`tweets_table`(`Date`,`Author`, `Text`, `Translate`, `Sentiment`, `Sensation`)VALUES(%s,%s,%s,%s,%s,%s)'''

        insert_tuple = (p_data,p_autor, p_texto, p_traducao, p_sentimento,p_sensacao)
        result = cursor.execute(sql_insert_query, insert_tuple)
        connection.commit()
        print ("Record inserted successfully into tweets_table table")
    except mysql.connector.Error as error :
        connection.rollback()
        print("Failed to insert into MySQL table {}".format(error))
    finally:
        #closing database connection.
        if(connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")


Once the function is finished it is time to start the connections with Twitter. 

I will not explain in this topic how to get the required key and token. This information can be obtained from the address below:

https://developer.twitter.com/

Here you will find all the information to create an account and get the key and token, as well as find all the API documentation for several languages like Python.

Next step is to pass the necessary connection information, all obtained from the site mentioned above. The information below is fake, it's only being used as an example.

In [ ]:
consumer_key = 'Baks8re9889rwe8r9we80r9we809r8q'
consumer_secret = '219uewqiueqwiur09wq898092hewqkj'
access_token = 'ifudsaoifsdafpsdaipofisdapipc098098q423'
access_secret = 'rwe87r0we88re09tre9twe9rd8937423ji'

Next step is to pass the information and make a connection.

In [ ]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [ ]:
print ("Downloading and Persisting tweets...")

Using a cursor we will pass the necessary parameters to the search API, such as the word to be searched, number of records per
page, whether to bring retweets or not and other fields that will be stored in the table as date, author, tweeted text and 
so on.

In [ ]:
for tweet in tweepy.Cursor(api.search,
                           q="word",
                           rpp=100,
                           tweet_mode='extended',
                           result_type="recent",
                           include_entities=True,
                           lang="pt-br").items(1000):
    if not tweet.retweeted and ('RT @' not in tweet.full_text):
        data = tweet.created_at
        autor = tweet.author.screen_name
        texto = tweet.full_text


For each processed tweet we will use the textblob to return the sentiment analysis. 

If we are working with other languages (in my case I was finding brasilian portuguese tweets), we can choose to translate the tweet into English and perform the feeling analysis considering that the English language is simpler and the algorithms more mature. Obviously, for very regional expressions we cannot get the correct analysis, but like the whole statistical algorithm, we will not hit 100% of the cases, but for most of the common expressions it works very well.

You can use textblob in order to make classifications (Naive Bayes and decision trees), tokenization, n-grams, word inflection, etc.

As the website itself says: TextBlob is a Python (2 and 3) library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.

More information of TextBlob can be found here: https://textblob.readthedocs.io/en/dev/


So each tweet is passed to the textblob that does the translation.


In [ ]:
frase = TextBlob(texto)
        translate = TextBlob(str(frase.translate(to='en')))

After we'll analyse the polarity and return the sentiment

In [ ]:
sentiment = translate.sentiment
        if sentiment.polarity < 0.0:
            sentiment2 = 'Negative'
        elif sentiment.polarity == 0.0:
            sentiment2 = 'Neutral'
        else:
            sentiment2 = 'Positive'
        

This time sleep is needed to prevent twitter from rating the program as a robot and blocking access to the API. 
this procedure is highly recommended. I learned after being blocked several times kkkkk

In [ ]:
time.sleep(10)

Finally you just get the time of the registry entry and write everything into the MySQL database.

In [ ]:
now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        insertInTable(now, author, text, str(translate), str(sentiment), sentiment2)

Below, the completed code.

In [ ]:
import tweepy
from tweepy import OAuthHandler
import timeimport pandas as pd
from textblob import TextBlob
import datetime
import mysql.connector
def insertInTable(p_date,p_author, p_text, p_translate, p_sentiment,p_sensation):
    try:
        connection = mysql.connector.connect(host='localhost',
                             database='database_name',
                             user='user',
                             password='password')
        cursor = connection.cursor(prepared=True)
        sql_insert_query = '''INSERT INTO `tweets`.`tweets_table`(`Date`,`Author`, `Text`, `Translate`, `Sentiment`, `Sensation`)VALUES(%s,%s,%s,%s,%s,%s)'''

        insert_tuple = (p_date,p_author, p_text, p_translate, p_sentiment,p_sensation)
        result = cursor.execute(sql_insert_query, insert_tuple)
        connection.commit()
        print ("Record inserted successfully into tweets_table table")
    except mysql.connector.Error as error :
        connection.rollback()
        print("Failed to insert into MySQL table {}".format(error))
    finally:
        #closing database connection.
        if(connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

for tweet in tweepy.Cursor(api.search,
                           q="word",
                           rpp=100,
                           tweet_mode='extended',
                           result_type="recent",
                           include_entities=True,
                           lang="pt-br").items(1000):
    if not tweet.retweeted and ('RT @' not in tweet.full_text):
        data = tweet.created_at
        autor = tweet.author.screen_name
        text = tweet.full_text
        phrase = TextBlob(texto)
        translate = TextBlob(str(phrase.translate(to='en')))
        sentiment = traducao.sentiment
        if sentiment.polarity < 0.0:
            sentiment2 = 'Negative'
        elif sentiment.polarity == 0.0:
            sentiment2 = 'Neutral'
        else:
            sentiment2 = 'Positive'
        time.sleep(10)
        print(author + "-" + text + "-" + str(sentiment) + "-" + str(sentiment2))
        now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        insertInTable(now, author, text, str(translate), str(sentiment), sentiment2)


With this few lines of code we can start storing a large amount of text for later use in other machine learning analysis.

You can use this code to make a spider and store web page content as well. You can find more information in this topic: A Little Snippet to Automate Web Scraping using Python and Selenium 

I hope you enjoyed the topic and see you next time!

Enjoy the code, improve it if you want to!

See you!!!!